# <strong><font color='red'>Identificación de tendencias y patrones relevantes.</font></strong>

## Librerías
Antes de comenzar a explorar nuestros datos, es importante importar las bibliotecas que nos permitirán manipularlos y visualizarlos. Utilizaremos pandas para trabajar con estructuras tipo DataFrame, numpy para funciones numéricas, y seaborn y matplotlib para gráficos.

In [23]:
# Importar librerías 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import 

SyntaxError: invalid syntax (1196730589.py, line 5)

## 🌡️ Tendencias y Patrones Relevantes en Temperatura

Datos reales de temperatura histórica en Chile, específicamente desde estaciones meteorológicas oficiales. Utilizaremos un conjunto de datos disponible públicamente en [GitHub](https://github.com/bastianolea/temperaturas_chile) que recopila temperaturas mínimas y máximas desde 1950 hasta 2024.

In [ ]:
# Ruta válida al archivo CSV
url = "https://raw.githubusercontent.com/bastianolea/temperaturas_chile/main/datos/procesados/temperaturas_chile.csv"

# Leer el archivo usando punto y coma como separador
df = pd.read_csv(url, sep=';')

# Limpieza básica
df.columns = df.columns.str.strip()  # Eliminar espacios en nombres de columnas
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

# Conversión de columna de fecha
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

df['t_min'] = pd.to_numeric(df['t_min'].str.replace(',', '.'), errors='coerce')
df['t_max'] = pd.to_numeric(df['t_max'].str.replace(',', '.'), errors='coerce')

# Crear columna de temperatura media
df['temperatura_media'] = (df['t_min'] + df['t_max']) / 2

# Verificar que todo esté bien
print(df[['fecha', 't_min', 't_max', 'temperatura_media']].head())

       fecha  t_min  t_max  temperatura_media
0 2013-01-01   16.7   23.9              20.30
1 2013-01-02   16.2   23.5              19.85
2 2013-01-03   20.1   24.3              22.20
3 2013-01-04   17.5   24.4              20.95
4 2013-01-05   19.8   23.9              21.85


In [30]:
# Filtrar por estación (ejemplo: Santiago)
df_santiago = df[df['nombre'].str.contains("Santiago", case=False)].copy()
df_santiago = df_santiago.dropna(subset=['fecha', 'temperatura_media'])

# Agrupar por mes y calcular promedio mensual
df_santiago['AñoMes'] = df_santiago['fecha'].dt.to_period('M')
df_mensual = df_santiago.groupby('AñoMes')['temperatura_media'].mean().reset_index()
df_mensual['Fecha'] = df_mensual['AñoMes'].dt.to_timestamp()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
# Visualizar evolución temporal
plt.figure(figsize=(14,6))
sns.lineplot(data=df_mensual, x='Fecha', y='temperatura_media', marker='o', color='darkorange')
plt.title("Temperatura Media Mensual - Santiago de Chile")
plt.xlabel("Fecha")
plt.ylabel("Temperatura (°C)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Descomposición de serie temporal
df_mensual.set_index('Fecha', inplace=True)
decomp = seasonal_decompose(df_mensual['temperatura_media'], model='additive', period=12)



In [ ]:
# Graficar componentes: tendencia, estacionalidad y residuales
decomp.plot()
plt.suptitle("Componentes de la Temperatura - Santiago", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:

# Identificar meses más cálidos o fríos
df_mensual['Mes'] = df_mensual.index.month
sns.boxplot(data=df_mensual, x='Mes', y='temperatura_media', palette='coolwarm')
plt.title("Distribución Mensual de Temperatura - Santiago")
plt.xlabel("Mes")
plt.ylabel("Temperatura (°C)")
plt.tight_layout()
plt.show()